In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

from sksurv.datasets import load_gbsg2
#from sksurv.preprocessing import OneHotEncoder
from sksurv.ensemble import RandomSurvivalForest


In [23]:
from sklearn.preprocessing import OneHotEncoder

In [2]:
clinical_mrna = pd.read_csv('clinical_data_subset/clinical_subset_mrna.csv')
clinical_meth = pd.read_csv('clinical_data_subset/clinical_subset_meth.csv')
clinical_mirna = pd.read_csv('clinical_data_subset/clinical_subset_mirna.csv')
clinical = pd.read_csv('clinical_data_subset/clinical_data.csv')

In [3]:
mrna = pd.read_csv('data/clean_datasets/mrna_clean.csv')
meth = pd.read_csv('data/clean_datasets/meth_clean.csv')
mirna = pd.read_csv('data/clean_datasets/mirna_clean.csv')

In [4]:
meth  = meth.drop(columns=['Unnamed: 0'])

In [5]:
def make_df(df):
    df = df.T
    headers = df.iloc[0]
    return pd.DataFrame(df.values[1:], columns=headers)

In [6]:
df_mrna = make_df(mrna)
df_meth = make_df(meth)
df_mirna = make_df(mirna)

In [7]:
df_mrna.shape, df_meth.shape, df_mirna.shape

((515, 20172), (458, 17052), (450, 477))

In [8]:
clinical.shape , clinical_mrna.shape, clinical_meth.shape, clinical_mirna.shape

((522, 10), (515, 10), (458, 10), (450, 10))

In [9]:
#mrna_meth subset
mrna_meth = list(set(clinical_mrna['sample'].tolist()) & set(clinical_meth['sample'].tolist()))
mrna_meth.sort()

In [10]:
#mrna_meth subset
mrna_mirna = list(set(clinical_mrna['sample'].tolist()) & set(clinical_mirna['sample'].tolist()))
mrna_mirna.sort()

In [11]:
#all 3 omics
mrna_meth_mirna =list(set(clinical_mrna['sample'].tolist()) & set(clinical_meth['sample'].tolist()) & set(clinical_mirna['sample'].tolist()))
mrna_meth_mirna.sort()

In [12]:
#clinical mrna meth
clinical_mrna_meth = clinical.loc[clinical['sample'].isin(mrna_meth)]
clinical_mrna_meth = clinical_mrna_meth.reset_index(drop=True)
clinical_mrna_meth = clinical_mrna_meth.set_index([mrna_meth])

In [13]:
#clinical_mrna_mirna and #clinical all omics are same
clinical_mrna_mirna = clinical.loc[clinical['sample'].isin(mrna_mirna)]
clinical_mrna_mirna = clinical_mrna_mirna.reset_index(drop=True)
clinical_mrna_mirna = clinical_mrna_mirna.set_index([mrna_mirna])

In [62]:
#del df_mrna_common, df_meth_common, df_mirna_common

In [63]:
#MRNA
df_mrna['sample'] = clinical_mrna['sample'].values
df_mrna_common = df_mrna[df_mrna['sample'].isin(mrna_meth)]
df_mrna_common = df_mrna_common.reset_index(drop=True)
df_mrna_common = df_mrna_common.set_index([mrna_meth])
df_mrna_common = df_mrna_common.drop(columns=['sample'])
#mrna common between mrna and meth completed

In [64]:
#METH
df_meth['sample'] = clinical_meth['sample'].values
df_meth_common = df_meth[df_meth['sample'].isin(mrna_meth)]
df_meth_common = df_meth_common.reset_index(drop=True)
df_meth_common = df_meth_common.set_index([mrna_meth])
df_meth_common = df_meth_common.drop(columns=['sample'])
#mrna common between mrna and meth completed

In [16]:
#starting for mirna
df_mirna['sample'] = clinical_mirna['sample'].values
df_mirna_common = df_mirna[df_mirna['sample'].isin(mrna_mirna)]
df_mirna_common = df_mirna_common.reset_index(drop=True)
df_mirna_common = df_mirna_common.set_index([mrna_mirna])
df_mirna_common = df_mirna_common.drop(columns=['sample'])
#mirna completed

In [65]:
df_mrna_common.columns = [str(col) + '_mrna' for col in df_mrna_common.columns]

In [66]:
df_meth_common.columns = [str(col) + '_meth' for col in df_meth_common.columns]

In [ ]:
#not required as mirna has mirna columns not genes
#df_mirna_common.columns = [str(col) + '_mirna' for col in df_mirna_common.columns]

In [67]:
df_meth_common.shape,df_mrna_common.shape, #df_mirna_common.shape

((454, 17052), (454, 20172))

In [68]:
dataset = pd.concat([df_mrna_common, df_meth_common], axis=1)

In [83]:
df_mrna = df_mrna.drop(columns=['sample'])
df_meth = df_meth.drop(columns=['sample'])
df_mirna = df_mirna.drop(columns=['sample'])

In [110]:
#del dataset
dataset = df_mrna

In [111]:
dataset

Gene,A1BG,A1CF,A2BP1,A2LD1,A2ML1,A2M,A4GALT,A4GNT,AAA1,AAAS,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR
0,-1.35187,-0.355983,0.252586,0.81819,-0.56545,-0.829673,-1.19649,-0.233423,-0.457862,0.0052995,...,-0.258269,-0.479972,-0.275024,-1.44764,0.913963,-0.597713,-0.362917,-0.390931,-1.13767,-0.33378
1,0.258303,-0.0700547,0.213372,-0.0880132,-0.125886,0.253316,0.715248,-0.488375,0.111104,-0.236638,...,0.761211,0.825834,0.392594,0.0509256,-0.191023,-0.966937,0.216038,0.166534,0.982386,-0.33378
2,-0.652402,-0.355983,-0.529276,1.05446,-0.764927,-0.217198,0.0911344,-0.263554,1.26307,-0.322381,...,0.681516,0.0752114,-0.689032,0.2034,-1.91685,0.678576,0.276884,-0.625059,-1.16745,-0.33378
3,0.465349,-0.355983,-0.529276,0.407735,0.404509,0.143436,0.660419,0.268695,1.35673,-0.215207,...,-0.221462,-0.055634,-0.317616,0.720355,0.296176,1.10889,0.27451,0.0470321,-0.0956753,-0.33378
4,0.320554,-0.355983,-0.529276,-0.124322,-0.764927,0.964868,-0.9988,0.143914,-0.457862,0.0785345,...,1.34341,1.15568,0.758608,-0.463072,-0.228734,-0.277139,1.13289,0.309396,1.4776,-0.33378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,-0.853336,-0.355983,-0.529276,0.285923,0.752226,0.0977948,1.35544,0.681954,-0.457862,-0.0117633,...,-0.19917,-0.18636,0.189037,-0.032048,-0.467175,0.164925,0.559791,-0.338681,1.71093,-0.33378
511,1.13806,3.88771,1.22577,-1.42343,-0.621465,0.351869,-0.455685,0.08043,2.54526,1.08099,...,0.294717,0.135591,1.1386,0.598593,-0.322182,-0.438477,1.42653,-0.202817,1.94186,2.68388
512,1.02302,-0.355983,1.62258,0.163188,0.115163,-1.76715,0.167545,-1.06599,-0.457862,-0.275452,...,-2.17529,-1.8473,0.515306,-0.33627,-1.53329,-1.28918,0.701999,-1.30371,-1.24571,-0.33378
513,0.851088,-0.355983,-0.00952321,-2.4434,-0.764927,0.274187,-2.25902,0.763506,-0.457862,0.424673,...,2.57552,1.15326,0.699177,-1.52971,-0.213705,0.299805,0.844005,-0.957366,1.00477,1.16159


In [ ]:
# dataset['time'] = clinical_mrna_mirna['os_time'].values
# dataset['event'] = clinical_mrna_mirna['vital_status'].values

In [112]:
#onehot clinical
enc = OneHotEncoder(handle_unknown='ignore')
clinical_data = clinical_mrna
one_hot_T = pd.DataFrame(enc.fit_transform(clinical_data[['pathology_T_stage']]).toarray(), columns=['tf1','tf2','tf3','tf4','tf5','tf6','tf7','tf8','tf9'])
one_hot_N = pd.DataFrame(enc.fit_transform(clinical_data[['pathology_N_stage']]).toarray(), columns=['nf1','nf2','nf3','nf4','nf5'])
one_hot_M = pd.DataFrame(enc.fit_transform(clinical_data[['pathology_M_stage']]).toarray(),
columns=['mf1','mf2','mf3','mf4','mf5'])
one_hot_G = pd.DataFrame(enc.fit_transform(clinical_data[['gender']]).toarray(),columns=['gf1','gf2'])

clinical_feat =pd.concat([one_hot_T,one_hot_N, one_hot_M, one_hot_G], axis=1)
clinical_feat = clinical_feat.reset_index(drop=True)

In [113]:
clinical_feat['age'] = clinical_data['age'].values

In [114]:
clinical_feat

,tf1,tf2,tf3,tf4,tf5,tf6,tf7,tf8,tf9,nf1,...,nf4,nf5,mf1,mf2,mf3,mf4,mf5,gf1,gf2,age
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,70
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,67
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,79
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,68
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,56
511,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,67
512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,49
513,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,74


In [115]:
dataset_X = pd.concat([dataset.reset_index(drop=True), clinical_feat.reset_index(drop=True)], axis=1)


In [116]:
dataset_X

,A1BG,A1CF,A2BP1,A2LD1,A2ML1,A2M,A4GALT,A4GNT,AAA1,AAAS,...,nf4,nf5,mf1,mf2,mf3,mf4,mf5,gf1,gf2,age
0,-1.35187,-0.355983,0.252586,0.81819,-0.56545,-0.829673,-1.19649,-0.233423,-0.457862,0.0052995,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,70
1,0.258303,-0.0700547,0.213372,-0.0880132,-0.125886,0.253316,0.715248,-0.488375,0.111104,-0.236638,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,67
2,-0.652402,-0.355983,-0.529276,1.05446,-0.764927,-0.217198,0.0911344,-0.263554,1.26307,-0.322381,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,79
3,0.465349,-0.355983,-0.529276,0.407735,0.404509,0.143436,0.660419,0.268695,1.35673,-0.215207,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,68
4,0.320554,-0.355983,-0.529276,-0.124322,-0.764927,0.964868,-0.9988,0.143914,-0.457862,0.0785345,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,-0.853336,-0.355983,-0.529276,0.285923,0.752226,0.0977948,1.35544,0.681954,-0.457862,-0.0117633,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,56
511,1.13806,3.88771,1.22577,-1.42343,-0.621465,0.351869,-0.455685,0.08043,2.54526,1.08099,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,67
512,1.02302,-0.355983,1.62258,0.163188,0.115163,-1.76715,0.167545,-1.06599,-0.457862,-0.275452,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,49
513,0.851088,-0.355983,-0.00952321,-2.4434,-0.764927,0.274187,-2.25902,0.763506,-0.457862,0.424673,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,74


In [29]:
# del dataset

In [ ]:
#dataset

In [117]:
#y = clinical_mrna.filter(['vital_status','os_time'], axis=1)
#y = clinical_mrna.filter(['vital_status','os_time'], axis=1)
y = clinical_mrna.filter(['vital_status','os_time'], axis=1)

In [118]:
y

,vital_status,os_time
0,0,0
1,0,1523
2,1,121
3,0,607
4,0,426
...,...,...
510,0,13
511,0,603
512,0,617
513,1,1798


In [119]:
y_struc = np.zeros(y.shape[0], dtype={'names':('vital_status','os_time'),'formats':('?','f8')})
y_struc['vital_status'] = y['vital_status'].astype('bool')
y_struc['os_time'] = y['os_time']

In [120]:
X = dataset_X.copy()

In [121]:
scores = []
for i in range(1,11):
    random_state = i
    X_train, X_test, y_train, y_test = train_test_split(X, y_struc, test_size=0.2,random_state=random_state)
    rsf = RandomSurvivalForest(n_estimators=100,min_samples_split=10,
                           min_samples_leaf=15,
                           max_features="sqrt",
                           n_jobs=-1,
                           random_state=random_state)
    rsf.fit(X_train, y_train)
    scores.append(rsf.score(X_test, y_test))

In [122]:
scores

[0.5626753975678204,
 0.6339285714285714,
 0.6737035337310693,
 0.6580286904211013,
 0.6104626026805016,
 0.6248548199767712,
 0.5808133472367049,
 0.562955254942768,
 0.6594594594594595,
 0.583409715857012]